In [1]:
from laplace import Laplace

In [13]:
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Load data from CSV file using pandas
df = pd.read_csv('data1.csv')

# Split the dataframe into x and y tensors
x = torch.from_numpy(df[['x1', 'x2']].to_numpy())
y = torch.from_numpy(df['y'].to_numpy(dtype=int))

In [27]:
X = torch.tensor(x.T).float().T

# Convert y to a tensor of indices and one-hot encode it
y_unique = torch.unique(torch.tensor(y))
y_indices = torch.searchsorted(y_unique, torch.tensor(y))
y_train = nn.functional.one_hot(y_indices, num_classes=len(y_unique)).float()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19352\1740726160.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(x.T).float().T
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19352\1740726160.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_unique = torch.unique(torch.tensor(y))
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19352\1740726160.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_indices = torch.searchsorted(y_unique, torch.tensor(y))


In [28]:
# Define neural network
n_hidden = 3
D = X.shape[1]
out_dim = y_train.shape[1]
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Sigmoid(),
    nn.Linear(n_hidden, out_dim)
)

In [29]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [31]:
# Create DataLoader for batch processing
batch_size = 10
data = TensorDataset(X, y_train)
loader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [36]:
# Train the model
epochs = 100
avg_loss = lambda data: nn.functional.cross_entropy(model(data[0]), torch.argmax(data[1], dim=1))
show_every = epochs // 10

for epoch in range(1, epochs+1):
    for x_batch, y_batch in loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, torch.argmax(y_batch, dim=1))
        loss.backward()
        optimizer.step()
    
    if epoch % show_every == 0:
        print("Epoch ", epoch)
        print("Avg loss: ", avg_loss((X, y_train)))


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19352\3827255648.py:6: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  for epoch in range(1, epochs+1):


Epoch  tensor(10.)
Avg loss:  tensor(0.0416, grad_fn=<NllLossBackward0>)
Epoch  tensor(20.)
Avg loss:  tensor(0.0398, grad_fn=<NllLossBackward0>)
Epoch  tensor(30.)
Avg loss:  tensor(0.0382, grad_fn=<NllLossBackward0>)
Epoch  tensor(40.)
Avg loss:  tensor(0.0366, grad_fn=<NllLossBackward0>)
Epoch  tensor(50.)
Avg loss:  tensor(0.0351, grad_fn=<NllLossBackward0>)
Epoch  tensor(60.)
Avg loss:  tensor(0.0337, grad_fn=<NllLossBackward0>)
Epoch  tensor(70.)
Avg loss:  tensor(0.0323, grad_fn=<NllLossBackward0>)
Epoch  tensor(80.)
Avg loss:  tensor(0.0310, grad_fn=<NllLossBackward0>)
Epoch  tensor(90.)
Avg loss:  tensor(0.0298, grad_fn=<NllLossBackward0>)
Epoch  tensor(100.)
Avg loss:  tensor(0.0286, grad_fn=<NllLossBackward0>)
